In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import concurrent.futures
from translate import Translator

In [2]:
ingredientes1k = pd.read_csv(r"top-1k-ingredients.csv", delimiter = ";")
ingredientes1k.head()

,nome,id
0,five spice powder,1002002
1,acorn squash,11482
2,adobo sauce,6979
3,agave nectar,19912
4,ahi tuna,15117


In [3]:
load_dotenv()
apikey = os.getenv('spoonacularApikey')

def obter_informacoes_produto(id_produto):
    url = f'https://api.spoonacular.com/food/ingredients/{id_produto}/information?amount=1&apiKey={apikey}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
    
def obter_informacoes_para_linha(row):
    return obter_informacoes_produto(row[1])

In [4]:
#Utilizar threading para pegar informações de cada produto, com sua id e api, colocando em um dataframe
#executor.map pega uma função e uma iteração, passando essa função por toda a iteração
#itertuples faz uma iteração do dataframe, com cada loop/(linha do dataframe) sendo uma tupla 
with concurrent.futures.ThreadPoolExecutor() as executor:
    resultados = list(executor.map(obter_informacoes_para_linha, ingredientes1k.head(30).itertuples(index=False)))

# Dataframe não tem a unidade relacionada ao valor estimado, logo, é preciso fazer um outro dataframe, utilizando uma das 
#possibleUnits (a primeira) e o id denovo, para assim ter o valor relacionado com a unidade 
dfSemUnit = pd.DataFrame(resultados)
dfSemUnit.head()

,id,original,originalName,name,amount,unit,unitShort,unitLong,possibleUnits,estimatedCost,consistency,aisle,image,meta,nutrition,categoryPath,shoppingListUnits
0,1002002,five spice powder,five spice powder,five spice powder,1.0,,,,"[g, oz, teaspoon, tablespoon]","{'value': 4.64, 'unit': 'US Cents'}",solid,Ethnic Foods;Spices and Seasonings,chinese-five-spice-powder.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[seasoning mix, seasoning]",NaN
1,11482,acorn squash,acorn squash,acorn squash,1.0,,,,"[squash, piece, g, oz, cup]","{'value': 94.82, 'unit': 'US Cents'}",solid,Produce,acorn-squash.jpg,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[winter squash, squash, vegetable]",[pieces]
2,6979,adobo sauce,adobo sauce,adobo sauce,1.0,,,,"[g, oz, teaspoon, cup, tablespoon]","{'value': 0.96, 'unit': 'US Cents'}",solid,Ethnic Foods,harissa.jpg,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...",[],NaN
3,19912,agave syrup,agave syrup,agave syrup,1.0,,,,"[g, oz, teaspoon, cup, serving, tablespoon]","{'value': 0.71, 'unit': 'US Cents'}",liquid,Ethnic Foods;Health Foods,agave.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...",[syrup],[ounces]
4,15117,ahi tuna steaks,ahi tuna steaks,ahi tuna steaks,1.0,,,,"[strip, steak, piece, g, ounce, fillet, inch, ...","{'value': 581.91, 'unit': 'US Cents'}",solid,Seafood,tuna-steak.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[tuna steak, fresh tuna, tuna, fish, main dish]","[ounces, pounds]"


In [5]:
#Deixar possibleUnits como primeira coluna
colunasOrdenadas = ['possibleUnits'] + [coluna for coluna in dfSemUnit.columns if coluna != 'possibleUnits']
dfSemUnit = dfSemUnit[colunasOrdenadas]

In [6]:
def obter_informacoes_produto_com_Unit(id_produto,unit):
    url = f'https://api.spoonacular.com/food/ingredients/{id_produto}/information?amount=1&apiKey={apikey}&unit={unit}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
    
def obter_informacoes_para_linha_com_Unit(row):
    return obter_informacoes_produto_com_Unit(row[1], row[0][0])

In [7]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    resultados = list(executor.map(obter_informacoes_para_linha_com_Unit, dfSemUnit.head(30).itertuples(index=False)))

# Criar um novo DataFrame com as informações dos produtos
dfComUnit = pd.DataFrame(resultados)
dfComUnit.head()

,id,original,originalName,name,amount,unit,unitShort,unitLong,possibleUnits,estimatedCost,consistency,aisle,image,meta,nutrition,categoryPath,shoppingListUnits
0,1002002,five spice powder,five spice powder,five spice powder,1.0,g,g,gram,"[g, oz, teaspoon, tablespoon]","{'value': 4.64, 'unit': 'US Cents'}",solid,Ethnic Foods;Spices and Seasonings,chinese-five-spice-powder.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[seasoning mix, seasoning]",NaN
1,11482,acorn squash,acorn squash,acorn squash,1.0,squash,squash,squash,"[squash, piece, g, oz, cup]","{'value': 94.82, 'unit': 'US Cents'}",solid,Produce,acorn-squash.jpg,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[winter squash, squash, vegetable]",[pieces]
2,6979,adobo sauce,adobo sauce,adobo sauce,1.0,g,g,gram,"[g, oz, teaspoon, cup, tablespoon]","{'value': 0.96, 'unit': 'US Cents'}",solid,Ethnic Foods,harissa.jpg,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...",[],NaN
3,19912,agave syrup,agave syrup,agave syrup,1.0,g,g,gram,"[g, oz, teaspoon, cup, serving, tablespoon]","{'value': 0.71, 'unit': 'US Cents'}",liquid,Ethnic Foods;Health Foods,agave.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...",[syrup],[ounces]
4,15117,ahi tuna steaks,ahi tuna steaks,ahi tuna steaks,1.0,strip,strip,strip,"[strip, steak, piece, g, ounce, fillet, inch, ...","{'value': 136.92, 'unit': 'US Cents'}",solid,Seafood,tuna-steak.png,[],"{'nutrients': [{'name': 'Vitamin B6', 'amount'...","[tuna steak, fresh tuna, tuna, fish, main dish]","[ounces, pounds]"


In [8]:
#Isola o valor da coluna estimatedCost, que tem ele dentro de um dicionário, depois usa deste valor para transformar em reais
#multiplicando por 4.93, dividindo por 100, arredondando em 2 casas decimais, e colocando em uma nova coluna Valor R$
dfComUnit['Valor'] = dfComUnit['estimatedCost'].apply(lambda x: x['value'])
dfComUnit['Valor R$'] = dfComUnit['Valor'].apply(lambda x: round((x * 4.93 / 100), 2))

In [9]:
def traduzir_texto(texto, idioma_destino='pt-BR'):
    translator = Translator(to_lang=idioma_destino)
    traducao = translator.translate(texto)
    return traducao

In [10]:
#tradução dos nomes de cada produto do ingês para português br
dfComUnit['Nome'] = dfComUnit['name'].apply(lambda x: traduzir_texto(x))

In [15]:
#Mudar nome de coluna de unitLong para Unidade
dfComUnit.rename(columns={'unitLong': 'Unidade'}, inplace=True)

In [24]:
#Ao ver o dataframe, um dos elementos ficou vazio, logo fazer essa correção
dfComUnit['Unidade'][11] = 'piece'

C:\Users\lucca\AppData\Local\Temp\ipykernel_4280\4001526339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfComUnit['Unidade'][11] = 'piece'


In [27]:
colunasDB = ['id','Nome','Valor R$','Unidade','image']
dfDB = dfComUnit[colunasDB]
dfDB

,id,Nome,Valor R$,Unidade,image
0,1002002,cinco especiarias em pó,0.23,gram,chinese-five-spice-powder.png
1,11482,Abóbora-bolota,4.67,squash,acorn-squash.jpg
2,6979,molho adobo,0.05,gram,harissa.jpg
3,19912,xarope de agave,0.04,gram,agave.png
4,15117,bifes de atum ahi,6.75,strip,tuna-steak.png
5,93606,molho macarrão alfredo,0.04,gram,alfredo-sauce.png
6,1002050,Extrato de amêndoa,0.09,drop,extract.png
7,93740,Amêndoas Moídas,0.10,gram,almond-meal-or-almond-flour.jpg
8,93607,bebida de amêndoa,0.01,gram,almond-milk.png
9,12061,amêndoa,2.64,handful,almonds.jpg


In [28]:
#Download dos dataframes 
dfDB.to_csv('SpoonacularApiDatasetSimples.csv', index=False)
dfComUnit.to_csv('SpoonacularApiDatasetCompleto.csv', index=False)